# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import ipywidgets as widgets
import math

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath="outputcityweather.csv"
csvread=pd.read_csv(csvpath)
df=pd.DataFrame(csvread)
df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Urengoy,69,RU,1596250298,61,65.97,78.37,289.79,5.02
1,Saint George,1,US,1596250298,10,37.10,-113.58,314.15,5.70
2,Ushuaia,75,AR,1596250022,65,-54.80,-68.30,278.15,6.20
3,Kavaratti,100,IN,1596250080,81,10.57,72.64,301.41,8.13
4,Torbay,90,CA,1596250300,100,47.67,-52.73,288.71,4.60
...,...,...,...,...,...,...,...,...,...
547,Ancud,90,CL,1596250403,93,-41.87,-73.82,282.59,5.70
548,Nueva Gerona,0,CU,1596250404,74,21.88,-82.80,302.27,8.95
549,Nanortalik,18,GL,1596250404,86,60.14,-45.24,279.44,5.87
550,Mandsaur,60,IN,1596250405,80,24.07,75.07,301.00,5.10


In [6]:
#conversion from Kelvin to Fahrenheit
Max_Temp = df["Max Temp"]
df["Temp in F"] = (Max_Temp - 273.15) * (9 / 5) + 32
df.head()



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Temp in Fahrenheit,Temp in F
0,Urengoy,69,RU,1596250298,61,65.97,78.37,289.79,5.02,61.952,61.952
1,Saint George,1,US,1596250298,10,37.10,-113.58,314.15,5.70,105.800,105.800
2,Ushuaia,75,AR,1596250022,65,-54.80,-68.30,278.15,6.20,41.000,41.000
3,Kavaratti,100,IN,1596250080,81,10.57,72.64,301.41,8.13,82.868,82.868
4,Torbay,90,CA,1596250300,100,47.67,-52.73,288.71,4.60,60.008,60.008


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
humidity=df["Humidity"]
maxhumidity=humidity.max()
longlatlocation=df[["Lat", "Lng"]]
mapfig = gmaps.figure()
heatmaplayer = gmaps.heatmap_layer(longlatlocation, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
mapfig.add_layer(heatmaplayer)
mapfig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
narrowe = df.loc[(df["Temp in F"] > 70) & (df["Temp in F"] < 80) & (df["Cloudiness"] == 0), :]
narrowe = narrowe.dropna(how='any')
narrowe.reset_index(inplace=True)
del narrowe['index']
narrowe.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Temp in Fahrenheit,Temp in F
0,Albany,0,US,1596250300,67,42.60,-73.97,296.48,1.64,73.994,73.994
1,Saskylakh,0,RU,1596250307,43,71.92,114.08,294.61,2.63,70.628,70.628
2,Guerrero Negro,0,MX,1596250082,72,27.98,-114.06,296.25,5.88,73.580,73.580
3,Fethiye,0,TR,1596250319,83,36.62,29.12,296.15,1.50,73.400,73.400
4,Wajir,0,KE,1596250331,74,1.75,40.06,294.88,4.21,71.114,71.114


In [15]:
hotel= []

for i in range(len(narrowe)):
    lat = narrowe.loc[i]['Lat']
    lng = narrowe.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params=params)
    response= request.json()
    try:
        hotel.append(response['results'][0]['name'])
    except:
        hotel.append("")


In [16]:
narrowe["Hotel Name"] = hotel
narrowe = narrowe.dropna(how='any')
narrowe.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Temp in Fahrenheit,Temp in F,Hotel Name
0,Albany,0,US,1596250300,67,42.60,-73.97,296.48,1.64,73.994,73.994,Voorheesville
1,Saskylakh,0,RU,1596250307,43,71.92,114.08,294.61,2.63,70.628,70.628,Saskylakh
2,Guerrero Negro,0,MX,1596250082,72,27.98,-114.06,296.25,5.88,73.580,73.580,Guerrero Negro
3,Fethiye,0,TR,1596250319,83,36.62,29.12,296.15,1.50,73.400,73.400,Fethiye
4,Wajir,0,KE,1596250331,74,1.75,40.06,294.88,4.21,71.114,71.114,Wajir


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowe.iterrows()]
locations = narrowe[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
marker=gmaps.marker_layer(locations)
mapfig.add_layer(marker)
mapfig


# Display figure


Figure(layout=FigureLayout(height='420px'))